<a href="https://colab.research.google.com/github/student-saptarshighosh/localrepo/blob/main/nstyletransfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

In [2]:
uploaded=files.upload()

Saving th.jpg to th.jpg


In [3]:
upload=files.upload()

Saving st.jpg to st.jpg


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import torchvision.transforms as tranforms
import torchvision.models as models
from torchvision.utils import save_image

model=models.vgg19(pretrained=True).features
#use 0,5,10,19,28 conv
#print(model)
class VGG(nn.Module):
    def __init__(self):
        super(VGG,self).__init__()
        self.chosen_feature=['0','5','10','19','28']
        self.model=models.vgg19(pretrained=True).features[:29]#upto 28 use in loss fuction
    def forward(self,x):
        features=[]
        for layer_num,layer in enumerate(self.model):
            x=layer(x)

            if str(layer_num) in self.chosen_feature:
                features.append(x)
        return features
def load_image(image_name):
    image=Image.open(image_name)
    image=loader(image).unsqueeze(0)#add an additional dim. for batch size which is just be 1 add thatdim. by unsqeeze
    return image.to(device)


device=torch.device("cuda" if torch.cuda.is_available else "cpu")
image_size=256


loader=tranforms.Compose(
    [
        tranforms.Resize((image_size,image_size)),
        tranforms.ToTensor(),
        #transform.Normalize(mean=[],std=[])
    ]
)
org_img=load_image("th.jpg")
sty_img=load_image("st.jpg")
model=VGG().to(device).eval()#here freeze the weight of generated
generated=org_img.clone().requires_grad_(True)
#or
#for noise generated
#generated=torch.randn(org_img.shape,device=device,requires_grad=True)

#hyperparametrs
total_step=6000
l_r=0.001
#here we take normalization not const. it divided by height,width,channel``
alpha=1
beta=0.01
optimizers=optim.Adam([generated],lr=l_r)

for step in range(total_step):
    generated_featrues=model(generated)
    org_img_features=model(org_img)
    sty_img_features=model(sty_img)
    style_loss=original_loss=0

    for gen_feature,org_feature,sty_feature in zip(
        generated_featrues,org_img_features,sty_img_features
    ):
        batch,ch,ht,wt=gen_feature.shape
        original_loss+=torch.mean((gen_feature-org_feature)**2)
        #compute gran matrix matrix multiplyw ith transpose i=ht,j=wt,k=ch,k'=transpose of ch #corelated matrix for generated img
        g=gen_feature.view(ch,batch*ht*wt).mm(
            gen_feature.view(ch,batch*ht*wt).t()
        )

        #compute gran matrix matrix multiplyw ith transpose i=ht,j=wt,k=ch,k'=transpose of ch #corelated matrix for generated img
        a=sty_feature.view(ch,batch*ht*wt).mm(
            sty_feature.view(ch,batch*ht*wt).t()
        )
        style_loss+=torch.mean((g-a)**2)
    total_loss=alpha*original_loss+beta*style_loss
    optimizers.zero_grad()
    total_loss.backward()
    optimizers.step()
    if step % 200 == 0:
        print(total_loss)
        save_image(generated,"gen.jpg")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:07<00:00, 76.3MB/s]


tensor(248977.1094, device='cuda:0', grad_fn=<AddBackward0>)
tensor(8697.6396, device='cuda:0', grad_fn=<AddBackward0>)
tensor(4572.4941, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3368.5200, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2769.8521, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2398.4478, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2148.5115, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1967.8184, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1830.9420, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1721.7296, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1630.4319, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1550.9703, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1479.4282, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1413.9519, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1352.7822, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1295.5453, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1241.2452, device='cuda:0', grad_fn=<AddBackwar